In [43]:
from sympy import Function, Derivative, Eq, symbols, sqrt
from sympkf import SymbolicPKF, t, FDModelBuilder

# Deriving the PKF dynamics for the GRS model with SymPKF

In [10]:
x = symbols("x")

k2,k4,k5,k6,l  = symbols("k_2 k_4 k_5 k_6 l")

E_ROC = Function("E_{ROC}")(x)
E_NO = Function("E_{NO}")(x)
E_NO2 = Function("E_{NO2}")(x)

u = Function("u")(x)  #wind
k1 = Function("k_1")(t)  #wind
k3 = Function("k_3")(t)  #wind

E_ROC = Function("E_{ROC}")(x) 
E_NO = Function("E_{NO}")(x) 
E_NO2 = Function("E_{NO2}")(x) 


#k1,k3 = symbols('k_1 k_3')
ROC = Function("ROC")(t,x)  
RP = Function("RP")(t,x)  
NO = Function("NO")(t,x)
NO2 = Function("NO2")(t,x)  
O3 = Function("O3")(t,x)  
SNGN = Function("SNGN")(t,x)  

#QSSA :
# RP = (k2*NO + 2*k6 *NO2)/(2*k5) * (sqrt(
#  1+ 4*k1*k5*ROC /(k2*NO+2*k6*NO2)**2
# )-1)


dynamics = [
    Eq(Derivative(ROC,t),-l*ROC + E_ROC -u*Derivative(ROC,x) -ROC*Derivative(u,x)) ,
    Eq(Derivative(RP,t),-l*RP +k1*ROC - RP * (k2*NO + 2*k6*NO2+ k5*RP)-u*Derivative(RP,x) -RP*Derivative(u,x)),
    Eq(Derivative(NO,t),-l*NO + E_NO + k3*NO2 - NO*(k2*RP+k4*O3) -u*Derivative(NO,x) -NO*Derivative(u,x)),
    Eq(Derivative(NO2,t),-l*NO2+ E_NO2 + k4*NO*O3 + k2*NO*RP - NO2*(k3 +2*k6*RP)-u*Derivative(NO2,x) -NO2*Derivative(u,x)),
    Eq(Derivative(O3,t),-l*O3+k3*NO2-k4*NO*O3-u*Derivative(O3,x) -O3*Derivative(u,x)),
    Eq(Derivative(SNGN,t),-l*SNGN+2*k6*NO2*RP-u*Derivative(SNGN,x) -SNGN*Derivative(u,x))
]

## GRS dynamics

In [11]:
for eq in dynamics : display(eq)

Eq(Derivative(ROC(t, x), t), -l*ROC(t, x) + E_{ROC}(x) - ROC(t, x)*Derivative(u(x), x) - u(x)*Derivative(ROC(t, x), x))

Eq(Derivative(RP(t, x), t), -l*RP(t, x) - (k_2*NO(t, x) + k_5*RP(t, x) + 2*k_6*NO2(t, x))*RP(t, x) + ROC(t, x)*k_1(t) - RP(t, x)*Derivative(u(x), x) - u(x)*Derivative(RP(t, x), x))

Eq(Derivative(NO(t, x), t), -l*NO(t, x) - (k_2*RP(t, x) + k_4*O3(t, x))*NO(t, x) + E_{NO}(x) - NO(t, x)*Derivative(u(x), x) + NO2(t, x)*k_3(t) - u(x)*Derivative(NO(t, x), x))

Eq(Derivative(NO2(t, x), t), k_2*NO(t, x)*RP(t, x) + k_4*NO(t, x)*O3(t, x) - l*NO2(t, x) - (2*k_6*RP(t, x) + k_3(t))*NO2(t, x) + E_{NO2}(x) - NO2(t, x)*Derivative(u(x), x) - u(x)*Derivative(NO2(t, x), x))

Eq(Derivative(O3(t, x), t), -k_4*NO(t, x)*O3(t, x) - l*O3(t, x) + NO2(t, x)*k_3(t) - O3(t, x)*Derivative(u(x), x) - u(x)*Derivative(O3(t, x), x))

Eq(Derivative(SNGN(t, x), t), 2*k_6*NO2(t, x)*RP(t, x) - l*SNGN(t, x) - SNGN(t, x)*Derivative(u(x), x) - u(x)*Derivative(SNGN(t, x), x))

## PKF for GRS 

In [12]:
pkf_dynamics = SymbolicPKF(dynamics)
for eq in pkf_dynamics.in_aspect : display(eq)

Eq(Derivative(ROC(t, x), t), -l*ROC(t, x) + E_{ROC}(x) - ROC(t, x)*Derivative(u(x), x) - u(x)*Derivative(ROC(t, x), x))

Eq(Derivative(RP(t, x), t), -k_2*NO(t, x)*RP(t, x) - k_2*V_RPNO(t, x) - k_5*RP(t, x)**2 - k_5*{V_{RP}}(t, x) - 2*k_6*NO2(t, x)*RP(t, x) - 2*k_6*V_RPNO2(t, x) - l*RP(t, x) + ROC(t, x)*k_1(t) - RP(t, x)*Derivative(u(x), x) - u(x)*Derivative(RP(t, x), x))

Eq(Derivative(NO(t, x), t), -k_2*NO(t, x)*RP(t, x) - k_2*V_RPNO(t, x) - k_4*NO(t, x)*O3(t, x) - k_4*V_NOO3(t, x) - l*NO(t, x) + E_{NO}(x) - NO(t, x)*Derivative(u(x), x) + NO2(t, x)*k_3(t) - u(x)*Derivative(NO(t, x), x))

Eq(Derivative(NO2(t, x), t), k_2*NO(t, x)*RP(t, x) + k_2*V_RPNO(t, x) + k_4*NO(t, x)*O3(t, x) + k_4*V_NOO3(t, x) - 2*k_6*NO2(t, x)*RP(t, x) - 2*k_6*V_RPNO2(t, x) - l*NO2(t, x) + E_{NO2}(x) - NO2(t, x)*k_3(t) - NO2(t, x)*Derivative(u(x), x) - u(x)*Derivative(NO2(t, x), x))

Eq(Derivative(O3(t, x), t), -k_4*NO(t, x)*O3(t, x) - k_4*V_NOO3(t, x) - l*O3(t, x) + NO2(t, x)*k_3(t) - O3(t, x)*Derivative(u(x), x) - u(x)*Derivative(O3(t, x), x))

...

Eq(Derivative({s_{RP,xx}}(t, x), t), -2*k_2*RP(t, x)*V_RPNO(t, x)*{s_{RP,xx}}(t, x)/{V_{RP}}(t, x) + 2*k_2*RP(t, x)*sqrt({V_{NO}}(t, x))*{s_{RP,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{NO}}(t, x, omega), x)*Derivative({\varepsilon_{RP}}(t, x, omega), x))/sqrt({V_{RP}}(t, x)) - k_2*RP(t, x)*sqrt({V_{NO}}(t, x))*{s_{RP,xx}}(t, x)**2*Expectation({\varepsilon_{NO}}(t, x, omega)*Derivative({\varepsilon_{RP}}(t, x, omega), x))*Derivative({V_{RP}}(t, x), x)/{V_{RP}}(t, x)**(3/2) + k_2*RP(t, x)*{s_{RP,xx}}(t, x)**2*Expectation({\varepsilon_{NO}}(t, x, omega)*Derivative({\varepsilon_{RP}}(t, x, omega), x))*Derivative({V_{NO}}(t, x), x)/(sqrt({V_{NO}}(t, x))*sqrt({V_{RP}}(t, x))) + 2*k_2*sqrt({V_{NO}}(t, x))*{s_{RP,xx}}(t, x)**2*Expectation({\varepsilon_{NO}}(t, x, omega)*Derivative({\varepsilon_{RP}}(t, x, omega), x))*Derivative(RP(t, x), x)/sqrt({V_{RP}}(t, x)) - 4*k_6*RP(t, x)*V_RPNO2(t, x)*{s_{RP,xx}}(t, x)/{V_{RP}}(t, x) + 4*k_6*RP(t, x)*sqrt({V_{NO2}}(t, x))*{s_{RP,xx}}(t, x)**2*

Eq(Derivative({s_{NO,xx}}(t, x), t), -2*k_2*NO(t, x)*V_RPNO(t, x)*{s_{NO,xx}}(t, x)/{V_{NO}}(t, x) + 2*k_2*NO(t, x)*sqrt({V_{RP}}(t, x))*{s_{NO,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{NO}}(t, x, omega), x)*Derivative({\varepsilon_{RP}}(t, x, omega), x))/sqrt({V_{NO}}(t, x)) + k_2*NO(t, x)*{s_{NO,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{NO}}(t, x, omega), x))*Derivative({V_{RP}}(t, x), x)/(sqrt({V_{NO}}(t, x))*sqrt({V_{RP}}(t, x))) - k_2*NO(t, x)*sqrt({V_{RP}}(t, x))*{s_{NO,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{NO}}(t, x, omega), x))*Derivative({V_{NO}}(t, x), x)/{V_{NO}}(t, x)**(3/2) + 2*k_2*sqrt({V_{RP}}(t, x))*{s_{NO,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{NO}}(t, x, omega), x))*Derivative(NO(t, x), x)/sqrt({V_{NO}}(t, x)) - 2*k_4*NO(t, x)*V_NOO3(t, x)*{s_{NO,xx}}(t, x)/{V_{NO}}(t, x) + 2*k_4*NO(t, x)*sqrt({V_{O3}}(t, x))*{s_{NO,xx}}(t, x)**2*Ex

Eq(Derivative({s_{NO2,xx}}(t, x), t), 2*k_2*NO(t, x)*V_RPNO2(t, x)*{s_{NO2,xx}}(t, x)/{V_{NO2}}(t, x) - 2*k_2*NO(t, x)*sqrt({V_{RP}}(t, x))*{s_{NO2,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{NO2}}(t, x, omega), x)*Derivative({\varepsilon_{RP}}(t, x, omega), x))/sqrt({V_{NO2}}(t, x)) - k_2*NO(t, x)*{s_{NO2,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{NO2}}(t, x, omega), x))*Derivative({V_{RP}}(t, x), x)/(sqrt({V_{NO2}}(t, x))*sqrt({V_{RP}}(t, x))) + k_2*NO(t, x)*sqrt({V_{RP}}(t, x))*{s_{NO2,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{NO2}}(t, x, omega), x))*Derivative({V_{NO2}}(t, x), x)/{V_{NO2}}(t, x)**(3/2) + 2*k_2*RP(t, x)*V_NONO2(t, x)*{s_{NO2,xx}}(t, x)/{V_{NO2}}(t, x) - 2*k_2*RP(t, x)*sqrt({V_{NO}}(t, x))*{s_{NO2,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{NO2}}(t, x, omega), x)*Derivative({\varepsilon_{NO}}(t, x, omega), x))/sqrt({V_{NO2}}(t, x)) - k_2*RP(t, x)*{s_{NO2,xx}}(t, x)**2*Expec

Eq(Derivative({s_{O3,xx}}(t, x), t), -2*k_4*O3(t, x)*V_NOO3(t, x)*{s_{O3,xx}}(t, x)/{V_{O3}}(t, x) + 2*k_4*O3(t, x)*sqrt({V_{NO}}(t, x))*{s_{O3,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{NO}}(t, x, omega), x)*Derivative({\varepsilon_{O3}}(t, x, omega), x))/sqrt({V_{O3}}(t, x)) - k_4*O3(t, x)*sqrt({V_{NO}}(t, x))*{s_{O3,xx}}(t, x)**2*Expectation({\varepsilon_{NO}}(t, x, omega)*Derivative({\varepsilon_{O3}}(t, x, omega), x))*Derivative({V_{O3}}(t, x), x)/{V_{O3}}(t, x)**(3/2) + k_4*O3(t, x)*{s_{O3,xx}}(t, x)**2*Expectation({\varepsilon_{NO}}(t, x, omega)*Derivative({\varepsilon_{O3}}(t, x, omega), x))*Derivative({V_{NO}}(t, x), x)/(sqrt({V_{NO}}(t, x))*sqrt({V_{O3}}(t, x))) + 2*k_4*sqrt({V_{NO}}(t, x))*{s_{O3,xx}}(t, x)**2*Expectation({\varepsilon_{NO}}(t, x, omega)*Derivative({\varepsilon_{O3}}(t, x, omega), x))*Derivative(O3(t, x), x)/sqrt({V_{O3}}(t, x)) + 2*V_NO2O3(t, x)*k_3(t)*{s_{O3,xx}}(t, x)/{V_{O3}}(t, x) - 2*k_3(t)*sqrt({V_{NO2}}(t, x))*{s_{O3,xx}}(t, x)**2*Expectation(

Eq(Derivative({s_{SNGN,xx}}(t, x), t), 4*k_6*NO2(t, x)*V_RPSNGN(t, x)*{s_{SNGN,xx}}(t, x)/{V_{SNGN}}(t, x) - 4*k_6*NO2(t, x)*sqrt({V_{RP}}(t, x))*{s_{SNGN,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{RP}}(t, x, omega), x)*Derivative({\varepsilon_{SNGN}}(t, x, omega), x))/sqrt({V_{SNGN}}(t, x)) + 2*k_6*NO2(t, x)*sqrt({V_{RP}}(t, x))*{s_{SNGN,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{SNGN}}(t, x, omega), x))*Derivative({V_{SNGN}}(t, x), x)/{V_{SNGN}}(t, x)**(3/2) - 2*k_6*NO2(t, x)*{s_{SNGN,xx}}(t, x)**2*Expectation({\varepsilon_{RP}}(t, x, omega)*Derivative({\varepsilon_{SNGN}}(t, x, omega), x))*Derivative({V_{RP}}(t, x), x)/(sqrt({V_{RP}}(t, x))*sqrt({V_{SNGN}}(t, x))) + 4*k_6*RP(t, x)*V_NO2SNGN(t, x)*{s_{SNGN,xx}}(t, x)/{V_{SNGN}}(t, x) - 4*k_6*RP(t, x)*sqrt({V_{NO2}}(t, x))*{s_{SNGN,xx}}(t, x)**2*Expectation(Derivative({\varepsilon_{NO2}}(t, x, omega), x)*Derivative({\varepsilon_{SNGN}}(t, x, omega), x))/sqrt({V_{SNGN}}(t, x)) + 2*k_6*RP(

## Simplification of the PKF dynamics

### cleaning covariance dynamics

In [32]:
mean_system = pkf_dynamics.in_aspect[:6]
var_system = pkf_dynamics.in_aspect[6:6+6]
cov_system= pkf_dynamics.in_aspect[6+6:6+6+15]
aspect_system = pkf_dynamics.in_aspect[6+6+15:]


new_cov_system = []
for eq in cov_system :
    new_eq = eq.subs(u,0).doit()
    covar = eq.args[0].args[0]
    new_eq = Eq(new_eq.args[0],new_eq.args[1]  -u *Derivative(covar,x) -2*covar*Derivative(u,x))
    new_cov_system.append(new_eq)

### simplifying aspect-tensor dynamics

In [41]:
new_aspect_system = []
for eq in aspect_system :
    variable = eq.args[0].args[0]
    new_eq = Eq(Derivative(variable,t), -u *Derivative(variable,x) +2*variable*Derivative(u,x))
    new_aspect_system.append(new_eq)

In [42]:
pkf_system = mean_system + var_system + new_cov_system + new_aspect_system

In [45]:
#saving generated classes :
FDModelBuilder(dynamics, class_name='adv_grs').write_module()
FDModelBuilder(pkf_system, class_name='pkf_adv_grs').write_module()

class adv_grs has been written in module adv_grs in file adv_grs.py
class pkf_adv_grs has been written in module pkf_adv_grs in file pkf_adv_grs.py


...